# Quickstart — Running Isaac 0.1
Spot shipping boxes inside the truck scene using the Isaac 0.1 model.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ericpence/perceptron_repo/blob/main/cookbook/quickstart/isaac/quickstart_isaac.ipynb)


## Install dependencies
Install the SDK plus helpers for environment loading and visualization.

In [ ]:
%pip install --upgrade perceptron pillow --quiet

## Configure the Perceptron client
Load your API key from the environment (or `.env`) and configure the SDK once for the rest of the notebook.

In [ ]:
from pathlib import Path

from IPython.display import Image as IPyImage
from IPython.display import display
from PIL import Image, ImageDraw

from cookbook.utils import cookbook_asset
from perceptron import configure, image, perceive, text

# configure() reads PERCEPTRON_API_KEY from the environment.
# configure() reads PERCEPTRON_API_KEY from the environment.
configure(
    provider="perceptron",
    # model="isaac-0.1",  # Enable once the SDK supports the model argument.
)

IMAGE_PATH = cookbook_asset("quickstart", "isaac", "truck_scene.jpg")
ANNOTATED_PATH = Path("truck_scene_annotated.jpg")

if not IMAGE_PATH.exists():
    raise FileNotFoundError(f"Expected sample image at {IMAGE_PATH}. Add it via cookbook/_shared/assets.")

## Detect shipping boxes
Wrap the detection prompt in a reusable `@perceive` helper, then call it on the sample frame.


In [ ]:
TARGET_CLASSES = ["boxes_in_the_truck"]


@perceive(expects="box", allow_multiple=True)
def detect_boxes(frame_path):
    scene = image(frame_path)
    friendly_classes = ", ".join(TARGET_CLASSES)
    return scene + text(
        f"Find every {friendly_classes}. Return one bounding box per instance and include a mention attribute."
    )


result = detect_boxes(str(IMAGE_PATH))

img = Image.open(IMAGE_PATH).convert("RGB")
draw = ImageDraw.Draw(img)


def to_px(point):
    return point.x / 1000 * img.width, point.y / 1000 * img.height


for idx, box in enumerate(result.points or []):
    top_left = to_px(box.top_left)
    bottom_right = to_px(box.bottom_right)
    draw.rectangle([top_left, bottom_right], outline="lime", width=3)
    label = box.mention or getattr(box, "label", None) or f"box {idx + 1}"
    draw.text((top_left[0], max(top_left[1] - 12, 0)), label, fill="lime")

img.save(ANNOTATED_PATH)
print("Annotated image saved to", ANNOTATED_PATH)

## Preview the annotated output
Display the saved overlay directly in the notebook so you can confirm grounding without leaving the runtime.

In [ ]:
if ANNOTATED_PATH.exists():
    display(IPyImage(filename=str(ANNOTATED_PATH)))
else:
    print("Run the detection cell first to generate the annotated image.")

## Conclusion & next steps
- Swap `TARGET_CLASSES` to label the objects you care about.
- Chain more cookbook capabilities (captioning, OCR, visual QA, in-context learning, tutorials) to expand the workflow.
- Convert the drawing logic into a reusable helper when you integrate with production tooling.
